追踪试验：记录您的配置参数
记录你的损失和指标
对数梯度和参数分布
记录你的模型
跟踪你的代码
记录您的系统指标（GPU、CPU、内存、温度等）

Make sure you set the project name before you initialize the Trainer.


project ：项目，一个experient，WANDB_PROJECT
Make sure you set the project name before you initialize the Trainer.初始化训练前，配置project-name

run：一个project中可以有很多run，

wandb.finish()结束， Jupyter 或 Google Colab notebook中使用

In [ ]:
last_run_id = "xxxxxxxx"  # fetch the run_id from your wandb workspace

# resume the wandb run from the run_id
with wandb.init(
    project=os.environ["WANDB_PROJECT"],
    id=last_run_id,
    resume="must",) as run:
    
  # Connect an Artifact to the run
  my_checkpoint_name = f"checkpoint-{last_run_id}:latest"
  my_checkpoint_artifact = run.use_artifact(my_model_name)
  
  # Download checkpoint to a folder and return the path
  checkpoint_dir = my_checkpoint_artifact.download()
  
  # reinitialize your model and trainer
  model = AutoModelForSequenceClassification.from_pretrained(
      <model_name>, num_labels=num_labels)
  # your awesome training arguments here.
  training_args = TrainingArguments(...) 
  
  trainer = Trainer(
      model=model,
      args=training_args,
      ...)
  
  # make sure use the checkpoint dir to resume training from the checkpoint
  trainer.train(resume_from_checkpoint=checkpoint_dir) 

In [ ]:
if __name__ == "__main__":
    # Get args
    args = parse_args()
    # Initialize run
    run = wandb.init(
        entity=args.entity,
        project=args.project,
        group="DDP",  # all runs for the experiment in one group
    )
    # Train model with DDP
    train(args, run)

In [ ]:
from accelerate import Accelerator

# Tell the Accelerator object to log with wandb
accelerator = Accelerator(log_with="wandb")

# Initialise your wandb run, passing wandb parameters and any config information
accelerator.init_trackers(
    project_name="my_project", 
    config={"dropout": 0.1, "learning_rate": 1e-2}
    init_kwargs={"wandb": {"entity": "my-wandb-team"}}
    )

...

# Log to wandb by calling `accelerator.log`, `step` is optional
accelerator.log({"train_loss": 1.12, "valid_loss": 0.8}, step=global_step)


# Make sure that the wandb tracker finishes correctly
accelerator.end_training()